In [11]:
# Import modules.
import torch
import random

import numpy as np

from gui import GUI
from hts import HTS
from network import H_GO
from torch import nn, optim
from collections import deque
from torchsummary import summary
from dataReader import DataReader
from trainer import SL_Trainer, RL_Trainer

In [12]:
# Set training device.
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available.")
else:
    device = torch.device("cpu")
    print("CUDA is not available, using CPU instead.")

CUDA is available.


In [13]:
# Set random seed.
random_seed = 0

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [14]:
# Set hyperparameters.
epochs = 10

batch_size = 10
data_num = 10
data_augment = False

In [15]:
# Load data.
data_reader = DataReader(dir_path="../dataset/10k", load_num=data_num, train_ratio=0.8, augment=data_augment)

Read raw data completed. (0.08 s)
Clear data completed. (0.00 s)
Convert data completed. (0.00 s)
Make train data completed. (0.01 s)
Split train and test data completed. (1616 train data, 404 test data).


In [16]:
# set model.
model = H_GO(input_size=8, output_size=361, hidden_dim=64)
model.to(device=device)

H_GO(
  (conv1): Conv2d(8, 64, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (tanh): Tanh()
  (layers): Sequential(
    (0): Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    

In [17]:
# Print model summary.
# summary(model, input_size=(8, 19, 19))

In [18]:
# Set optimizer and loss function.
optimizer = optim.RAdam(params=model.parameters())
loss_fu_policy = nn.CrossEntropyLoss()
loss_fu_value = nn.BCELoss()

In [19]:
sl_trainer = SL_Trainer(epochs=epochs, model=model, device=device, optimizer=optimizer, loss_fu_policy=loss_fu_policy, loss_fu_value=loss_fu_value)
sl_trainer.train(data_reader=data_reader, data_num=data_num, batch_size=batch_size)

Epoch: 000 | Loss of model: 6.498 | Accuracy of policy: 0.000% | Accuracy of value: 40.000%
Epoch: 001 | Loss of model: 6.562 | Accuracy of policy: 0.000% | Accuracy of value: 90.000%
Epoch: 002 | Loss of model: 6.585 | Accuracy of policy: 0.000% | Accuracy of value: 70.000%
Epoch: 003 | Loss of model: 6.494 | Accuracy of policy: 0.000% | Accuracy of value: 70.000%
Epoch: 004 | Loss of model: 6.737 | Accuracy of policy: 0.000% | Accuracy of value: 80.000%
Epoch: 005 | Loss of model: 6.597 | Accuracy of policy: 10.000% | Accuracy of value: 20.000%
Epoch: 006 | Loss of model: 6.506 | Accuracy of policy: 0.000% | Accuracy of value: 80.000%
Epoch: 007 | Loss of model: 6.474 | Accuracy of policy: 0.000% | Accuracy of value: 90.000%
Epoch: 008 | Loss of model: 6.718 | Accuracy of policy: 0.000% | Accuracy of value: 90.000%
Epoch: 009 | Loss of model: 6.679 | Accuracy of policy: 0.000% | Accuracy of value: 80.000%


In [24]:
def play(model: H_GO) -> None:
    # Set model to evaluation mode.
    model.eval()

    # Set defualt value.
    player = -1
    game_board = np.zeros((19, 19))
    game_queue = deque(maxlen=7)
    gui = GUI(size=19, label_mode="char")

    for _ in range(7):
        game_queue.append(game_board.copy())

    while True:
        best_move = HTS(state=game_queue, depth=4, breadth=4, temperature=0.5, player=player, model=model, device=device).get_best_move()

        row, col = best_move // 19, best_move % 19
        game_board[row][col] = player
        game_queue.append(game_board.copy())

        print(f"AI's move: ({row}, {col})")
        gui.display(game_board, best_move)
        print()

        player_move = input()

        if player_move == "q": break

        row, col = player_move.split(",")
        row, col = int(row), int(col)
        game_board[row][col] = 1
        game_queue.append(game_board.copy())

        print(f"Player's move: ({row}, {col})")
        gui.display(game_board, row * 19 + col)
        print()

In [26]:
play(model=model)

AI's move: (10, 2)
  A B C D E F G H I J K L M N O P Q R S
A · · · · · · · · · · · · · · · · · · · 
B · · · · · · · · · · · · · · · · · · · 
C · · · · · · · · · · · · · · · · · · · 
D · · · · · · · · · · · · · · · · · · · 
E · · · · · · · · · · · · · · · · · · · 
F · · · · · · · · · · · · · · · · · · · 
G · · · · · · · · · · · · · · · · · · · 
H · · · · · · · · · · · · · · · · · · · 
I · · · · · · · · · · · · · · · · · · · 
J · · · · · · · · · · · · · · · · · · · 
K · ·(X)· · · · · · · · · · · · · · · · 
L · · · · · · · · · · · · · · · · · · · 
M · · · · · · · · · · · · · · · · · · · 
N · · · · · · · · · · · · · · · · · · · 
O · · · · · · · · · · · · · · · · · · · 
P · · · · · · · · · · · · · · · · · · · 
Q · · · · · · · · · · · · · · · · · · · 
R · · · · · · · · · · · · · · · · · · · 
S · · · · · · · · · · · · · · · · · · · 

